# ipl I317B Sécurité : labos
## Semaine 1 : introduction et outils python

### Partie 1 : manipuler du texte

#### Exercice 1
Pour ce premier exercice, nous allons manipuler un court extrait de logs d'un serveur web nginx.  
Pour chaque ligne de celui-ci, extrayez:
- l'adresse IP
- le timestamp
- le chemin demandé

In [23]:
logs = """91.160.48.68 - - [3/Sep/2020:18:18:10 +0200] "GET /?C=M;O=D HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
78.224.72.158 - - [3/Sep/2020:18:24:52 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
192.241.199.4 - - [3/Sep/2020:19:11:15 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 zgrab/0.x" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "GET /.env HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "POST / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
185.172.110.223 - - [3/Sep/2020:19:16:37 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0" "-"
66.249.75.16 - - [3/Sep/2020:19:23:28 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-"
66.249.75.23 - - [3/Sep/2020:19:23:30 +0200] "GET / HTTP/1.1" 200 0 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-" """

In [35]:
for line in logs.split("\n"):
    print(line.split(" - - ",1)[0])
    print(line.split(" - - ",1)[1][:27].strip("[]"))
    print(line.split(" - - ",1)[1].split(" ")[3])
    print("----")

91.160.48.68
3/Sep/2020:18:18:10 +0200
/?C=M;O=D
----
78.224.72.158
3/Sep/2020:18:24:52 +0200
/
----
192.241.199.4
3/Sep/2020:19:11:15 +0200
/
----
40.121.69.161
3/Sep/2020:19:16:32 +0200
/.env
----
40.121.69.161
3/Sep/2020:19:16:32 +0200
/
----
185.172.110.223
3/Sep/2020:19:16:37 +0200
/
----
66.249.75.16
3/Sep/2020:19:23:28 +0200
/
----
66.249.75.23
3/Sep/2020:19:23:30 +0200
/
----


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [70]:
logs_set = set()
for line in logs.split("\n"):
    #print(line.split('"-"')[1][2:-2])
    logs_set.add(line.split('"-"')[1][2:-2])
print("Different user agent: ", len(logs_set))

#en comptant les occurences de chacun


logs_map = {}

for line in logs.split("\n"):
    ua = line.split('"-"')[1][2:-2]
    
    if ua not in logs_map:
        logs_map[ua] = 0
    
    logs_map[ua] += 1;

print("---------")
for k, v in logs_map.items():
    print(v, '=>', k)

Different user agent:  5
---------
2 => Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0
1 => Mozilla/5.0 zgrab/0.x
2 => Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36
1 => Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0
2 => Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)


### Partie 2 : requests et BeautifulSoup

Assurez vous que les paquets suivant soient bien installé :

`pip3 install --user requests bs4`

`sudo pip3 install requests bs4`

Depuis un jupyter notebook, vous pouvez probablement juste executer la case suivante :

In [88]:
!pip3 install --user requests bs4

Le packet requests nous permet de faire des requêtes web facilement, par exemple :

In [90]:
import requests

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")

if ipl_req.ok: # ipl_req.status_code == 200 c'est la meme chose
    ipl_html = ipl_req.text
    print(ipl_req.status_code)
    
else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

# Doc requests : https://requests.readthedocs.io/en/latest/ )

200


Nous pouvons ensuite lire le code source de la page en affichant `ipl_html`.

Parser du html avec des split ou des regex deviendrait vite infernal, c'est là qu'intervient la librairie BeautifulSoup qui nous permet de parcourir beaucoup plus simplement le dom notamment avec les fonctions `find` et `find_all`.

In [100]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(ipl_html)

# pour récupérer le contenu de la balise <title> :
title = soup.find("title").get_text()

print(title)

Informatique de gestion | HE VINCI


La fonction `find` et la fonction `find_all` vous renverons un object de type `tag` (balise) sur lequel vous pouvez faire :
  * d'autres `find()` et `find_all()`
  * `get_text()` pour récupérer tout le contenu textuel de la balise
  * `["nom de l'attribut"]` pour récupérer un attribut en particulier (exemple: `tag['href']`)
  * ...

( Doc BeautifulSoup4 : https://www.crummy.com/software/BeautifulSoup/bs4/doc/ )

#### Exercice 1:
Récupérez le contenue de la page suivante et affichez son titre proprement : https://www.vinci.be/fr/formations/informatique-de-gestion

In [101]:
from bs4 import BeautifulSoup
import requests

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")

soup = BeautifulSoup(ipl_html)

# pour récupérer le contenu de la balise <title> :
title = soup.find("title").get_text()

print(title)

Informatique de gestion | HE VINCI


Il est possible de faire des requêtes beaucoup plus poussée en ajoutant des arguments correspondant aux attributs des balises que nous cherchons, par exemple pour trouver les balises `<div class="footer"></div>` :

```
soup.find_all("div", class_="footer")
# NB: class est un mot clé réservé en python, d'où l'ajout du "_" dans BeautifulSoup
```

Ou encore pour trouver tous les liens vers la page facebook de l'ipl :

`soup.find_all("a", href="https://www.facebook.com/HEVINCI")`

BeautifoulSoup accepte également des regex compilée en argument, il est donc possible de récupérer la liste des url qui ne sont pas du domaines de "vinci.be" :

```
import re

not_vinci = re.compile("^http(s?)://(?!www.vinci.be/).*$")
soup.find_all("a", href=not_vinci)
```

Il est également possible d'utiliser une fonction arbitraire pour valider les liens. Par exemple, pour trouver tous les liens de moins de 30 caractères :

```
def less_than_30(tag):
    return len(tag) < 30  # on retourne un booleen
    
# on passe notre fonction en paramètre sans l'executer :
soup.find_all("a", href=less_than_30)
```

#### Exercice 2:

Utilisez des regex pour récupérer la liste des liens http (et donc pas https) sur la page :  
https://www.vinci.be/fr/formations/informatique-de-gestion

In [102]:
ipl_https = requests.get("https://www.vinci.be/fr/formations/informatique-de-gestion")
soup = BeautifulSoup(ipl_https.text)

In [46]:
import re



#### Exercice 3:

Utilisez une fonction pour récupérer tous les liens qui finissent avec une extension désignant un type de ficher (ex: ".html", ".php", ".pdf", ...) sur la page :  
https://www.vinci.be/fr/formations/informatique-de-gestion

In [47]:
#TODO

#### Exercice 4:
Depuis le site du Centre Cybersecurité Belgique https://ccb.belgium.be/fr/actualit%C3%A9s , récupérez la liste des articles récents et affichez leurs titres ainsi qu'un lien vers l'article.

In [110]:
req = requests.get("https://ccb.belgium.be/fr/actualit%C3%A9s")
soup = BeautifulSoup(req.text)

print(req.status_code)

for article in soup.find_all("article"):
    print(article.find('h2', class_ = "teaser-title"))
    
    break

200
<h2 class="teaser-title">10 talents belges ont participé à la compétition European Cybersecurity Challenge</h2>


#### Exercice 5:
La lib requests permet également d'effectuer de requête POST. Pour tester cette fonctionnalitée, créez un "requestbin" sur le site http://requestbin.net/ puis utilisez requests pour poster des données vers votre requestbin.

> RequestBin gives you a URL that will collect requests made to it and let you inspect them in a human-friendly way.
Use RequestBin to see what your HTTP client is sending or to inspect and debug webhook requests.


Voir : https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request (ou la cheatsheet)

In [49]:
#TODO

#### Exercice 6:

Maintenant que vous maîtrisez le get, le post et quelques bases de parsing web, "connectez-vous" au formulaire suivant avec les identifiants : `admin/admin`  
http://labosecuipl.alwaysdata.net/21/s01/form/

In [50]:
#TODO

#### Exercice 7:
Vous avez certainement constaté sur votre requestbin que la librairie requests utilise par défaut un User-Agent explicite "*je suis un lib python pour faire des requêtes*".  
Documentez-vous sur la manière de changer celui-ci dans requests et remplacez le dans votre précédente requête vers requestbin par quelque chose de plus discret, par exemple l'user-agent de la dernière version de chrome :  
`Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36`

source: https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome

In [51]:
#TODO

# Partie 3 (bonus):

Note: dans les jupyter notebook, vous pouvez exécuter des commandes shell en préfixant une cellule d'un `!`. Par exemple : `!date`.

Exercice 1

Pour ce premier exercice, nous allons manipuler un court extrait de logs "logs.txt" d'un serveur web nginx (le même que pour la partie 1).

Pour chaque ligne de celui-ci, extrayez:

    l'adresse IP
    le timestamp
    le chemin demandé

En n'utilisant que des commandes bash : cat, grep, cut, sort, uniq,...

In [52]:
!date

Tue Sep 20 05:21:27 AM UTC 2022


In [53]:
!echo "TODO" | grep "DO"

TODO


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [54]:
!echo "TODO"

TODO
